### An ensemble of Neural networks to classify the Fashion MNIST images

In [3]:
from keras.models import load_model
from keras.datasets import fashion_mnist


import numpy as np
import matplotlib.pyplot

### Load the fasion MNIST data

In [4]:
# Load the training and testing data
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
# Display purpose:
X_train_orig = X_train
X_test_orig = X_test

In [6]:
from keras import backend as K
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    shape_ord = (1, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 1)
    

In [7]:
X_train = X_train.reshape((X_train.shape[0],) + shape_ord)
X_test = X_test.reshape((X_test.shape[0],) + shape_ord)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

## Load the NN models that are already trained

In [8]:
cnn_model = load_model('cnn.h5')
mlff_bp_model = load_model('mlff_bp_model.h5')

In [10]:
slice = 100

models = {"CNN": cnn_model, "MLFF BP": mlff_bp_model}
accuracies = [0.9, 0.88]
test_data = X_test[:slice]

prediction_results = []

for model_name, model in models.items():
    prediction_results.append((model_name, model.predict(test_data).argmax(-1)))
    
prediction_results

[('CNN',
  array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5,
         7, 5, 1, 2, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 6, 7,
         6, 7, 2, 1, 2, 6, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1,
         2, 4, 7, 8, 7, 0, 2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2,
         0, 2, 5, 3, 6, 7, 1, 8, 0, 1, 2, 2], dtype=int64)),
 ('MLFF BP',
  array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 5, 3, 4, 1, 2, 2, 8, 0, 2, 5,
         7, 5, 1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 0, 1, 6, 7,
         6, 7, 2, 1, 2, 6, 4, 2, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1,
         3, 3, 7, 8, 7, 0, 2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2,
         0, 2, 5, 3, 6, 7, 1, 8, 0, 1, 4, 2], dtype=int64))]